# PySWMM - Python API to SWMM

This notebook demonstrates the basic capability provided by the PySWMM API to EPA SWMM. The basic example uses a simple tutorial model developed in PCSWMM, contained for present purposes in the `pcswmm_ex01/example01.inp` file. The PySWMM API can be used in conjuction with professional grade model development applications like PCSWMM to enable programmatic control of stormwater collection system models.

## Outline
The basic functionalities demonstrated in this notebook include programmatic manipulation of the previously created SWMM model to:

-  Read model output (from PCSWMM runs) in Jupyter
-  Write changes to model input files from Jupyter
-  Run the SWMM model programatically from Jupyter
-  Generate simple interactive graphics summarizing impacts of changes

### Read model output from prior runs

In [1]:
# viz imports
import plotly.graph_objects as go 
from dash import Dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
# quant imports
import pyswmm
import swmmio
from swmm.toolkit.shared_enum import NodeAttribute
import random
import pandas as pd
import numpy as np

In [ ]:
# Build App
app = Dash(__name__)
app.layout = html.Div([
    html.H1("SWMM Model Updating"),
    dcc.Graph(id='graph'),
    html.Button(id='rain-button', n_clicks=0, children='New Rainfall')
])
    
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Input('rain-button','n_clicks')
)
def update_rain(n_clicks, file_cnt=2):
    # get current model rainfall
    baseline = swmmio.Model('swmm_files/example1.inp')
    rain = baseline.inp.timeseries.Value.values

    # induce noise
    new_rain = np.array([random.expovariate(0.5)*float(i) for i in rain])
    baseline.inp.timeseries.Value = new_rain
    # save back to new model input file
    basepath = 'pcswmm_ex01/example'
    filext = '.inp'
    newfp = basepath + str(n_clicks+2) + filext
    baseline.inp.save(newfp)
    # construct name for new output file
    newop = basepath+str(n_clicks+2)+'.out'
    # execute new simulation
    with pyswmm.Simulation(newfp) as sim:
        sim.execute()
    # return updated figure from new output file
    return update_figure(newop)

def update_figure(newop):
    # read original model output
    with pyswmm.Output('pcswmm_ex01/example1.out') as out:
        ts0 = pd.Series(out.node_series(
            'OF1',
            NodeAttribute.INVERT_DEPTH
        ))

    # read new model output
    with pyswmm.Output(newop) as out:
        ts1 = pd.Series(out.node_series(
            'OF1',
            NodeAttribute.INVERT_DEPTH
        ))

    # Create figure
    fig = go.Figure()
    fig.update_layout(title_text="Depth of Flow at OF1")
    fig.add_trace(go.Line(x=ts0.index,y=ts0.values,
                        mode='lines',
                        name='original'))
    fig.add_trace(go.Line(x=ts1.index,y=ts1.values,
                        mode='lines',
                        name=newop))
    fig.update_layout(transition_duration=100)
    return fig

# Run app and display result inline in the notebook
app.run_server()


... SWMM Version 5.2.4
... Run Complete

c:\temp\swmm-dash-mlflow\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.





... SWMM Version 5.2.4
... Run Complete

c:\temp\swmm-dash-mlflow\.venv\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


[2023-12-01 15:29:05,866] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "c:\temp\swmm-dash-mlflow\.venv\Lib\site-packages\flask\app.py", line 1455, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\temp\swmm-dash-mlflow\.venv\Lib\site-packages\flask\app.py", line 869, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\temp\swmm-dash-mlflow\.venv\Lib\site-packages\flask\app.py", line 867, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\temp\swmm-dash-mlflow\.venv\Lib